In [1]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import reuters
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from collections import Counter

import gensim
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

import logging

stop_words = stopwords.words("english")

In [2]:
print("The reuters corpus has {} tags".format(len(reuters.categories())))
print("The reuters corpus has {} documents".format(len(reuters.fileids())))

The reuters corpus has 90 tags
The reuters corpus has 10788 documents


In [3]:
categories = []
file_count = []

for i in reuters.categories():
    file_count.append(len(reuters.fileids(i)))
    categories.append(i)

cats = pd.DataFrame({'categories': categories, "file_count": file_count}).sort_values('file_count', ascending=False)
cats.head()

,categories,file_count
21,earn,3964
0,acq,2369
46,money-fx,717
26,grain,582
17,crude,578


##### chose the second and third tags on this top tags list, since the first earn tag is most likely the highly-standardized news pieces with earnings reports.

#### Select documents that only contains top two labels with most documents

In [4]:
cat_start = 1
cat_end = 2
category_filter = cats.iloc[cat_start:cat_end + 1, 0].values.tolist()
category_filter

['acq', 'money-fx']

In [5]:
# select fileid with the category filter
doc_id_list = np.array(reuters.fileids(category_filter))
doc_id_list = doc_id_list[doc_id_list != 'training/3267']

In [6]:
train_labels = [reuters.categories(doc_id)for doc_id in doc_id_list if 'training' in doc_id]

In [7]:
test_labels = [reuters.categories(doc_id)for doc_id in doc_id_list if 'test' in doc_id]

In [8]:
train_doc = doc_id_list[['training' in doc_id for doc_id in doc_id_list]]
print("train_doc is created with following document names: {} ...".format(train_doc[0:5]))

train_doc is created with following document names: ['training/10' 'training/1000' 'training/10005' 'training/10018'
 'training/10025'] ...


In [9]:
test_doc = doc_id_list[['test' in doc_id for doc_id in doc_id_list]]
print("test_doc is created with following document names: {} ...".format(test_doc[0:5]))

test_doc is created with following document names: ['test/14843' 'test/14849' 'test/14852' 'test/14861' 'test/14865'] ...


In [10]:
test_corpus = [" ".join([t for t in reuters.words(test_doc[t])])
               for t in range(len(test_doc))]
print("test_corpus is created, the first line is: {} ...".format(test_corpus[0][:100]))

test_corpus is created, the first line is: SUMITOMO BANK AIMS AT QUICK RECOVERY FROM MERGER Sumitomo Bank Ltd & lt ; SUMI . T > is certain to l ...


In [11]:
train_corpus = [" ".join([t for t in reuters.words(train_doc[t])])
                for t in range(len(train_doc))]
print("train_corpus is created, the first line is: {} ...".format(train_corpus[0][:100]))

train_corpus is created, the first line is: COMPUTER TERMINAL SYSTEMS & lt ; CPML > COMPLETES SALE Computer Terminal Systems Inc said it has com ...


In [12]:

def load_response(dataframe, col_name):
    """
    Function to load survey response from a pandas dataframe into a list
    object that can be passed to the clean_corpus() function
    -----PARAMETERS-----
    dataframe: the pandas dataframe where the survey responses are stored
    col_name: a string of the column name of the survey responses
    -----OUTPUT-----
    Returned object is a list of responses as strings
    """
    try:
        responses = [t[0] for t in dataframe[[col_name]].values.tolist()]
    except (TypeError, NameError):
        print("Please input string as col_name")
        pass
    return responses


def clean_corpus(texts, string_line=True, stopping=True, pos='v'):
    """
    Function to clean up survey answers and return list for NLP processing
    --------PARAMETERS---------
    texts: list objects that contains survey response strings
    string_line: if True, each returned survey response is a single string
    if False, each response is a list of words in the original sequence
    stopping: (default) if True, filter stopwords
    pos: (default) if 'v', lemmatize input words as verbs;
    if 'n', lemmatize input words as nouns
    """
    cleaned = []
    i = 0
    stop = set(stopwords.words("english"))
    # print("$$$ empty cleaned created")
    print(">>>> response cleaning initiated")
    for text in texts:
        if (i + 1) % 500 == 0:
            print("--cleaning response #{} out of {}".format(i + 1, len(texts)))
        try:
            text = re.sub("[^a-zA-Z]", " ", text)
            text = word_tokenize(text)
            text = [t.lower() for t in text]
            if stopping:
                text = [t for t in text if t not in stop]
            lemmatizer = WordNetLemmatizer()
            text = [lemmatizer.lemmatize(t, pos=pos) for t in text]
            # TODO: determine which lemmatizer to use for this project
            cleaned.append(text)
        except TypeError:
            cleaned.append([])
        i += 1
    if string_line:
        cleaned = [" ".join(t) for t in cleaned]
    return cleaned


def get_bow(tokenized_text):
    """
    Function to generate bow_list and word_freq from a tokenized_text
    -----PARAMETER-----
    tokenized_text should be in the form of [['a'], ['a', 'b'], ['b']] format,
    where the object is a list of survey response, with each survey response
    as a list of word tokens
    -----OUTPUT-----
    The function returns two objects
    bow_list: a list of Counter objects with word frequency of each response
    word_freq: a Counter object that summarizes the word frequency of the input
    tokenized_text
    """
    bow_list = []
    word_freq = Counter()
    for text in tokenized_text:
        bow = Counter(text)
        word_freq.update(text)
        bow_list.append(bow)
    print("This corpus has {} key words, and the 10 \
most frequent words are: {}".format(len(word_freq.keys()), word_freq.most_common(10)))
    return bow_list, word_freq

In [13]:
# create clean corpus for word2vec approach
test_clean_string = clean_corpus(test_corpus)
train_clean_string = clean_corpus(train_corpus)
print('The first few words from cleaned test_clean_string is: {}'.format(test_clean_string[0][:100]))
print('The first few words from cleaned train_clean_string is: {}'.format(train_clean_string[0][:100]))

>>>> response cleaning initiated
--cleaning response #500 out of 898
>>>> response cleaning initiated
--cleaning response #500 out of 2186
--cleaning response #1000 out of 2186
--cleaning response #1500 out of 2186
--cleaning response #2000 out of 2186
The first few words from cleaned test_clean_string is: sumitomo bank aim quick recovery merger sumitomo bank ltd lt sumi certain lose status japan profitab
The first few words from cleaned train_clean_string is: computer terminal systems lt cpml complete sale computer terminal systems inc say complete sale shar


##### After the text is cleaned, I can now apply the BOW model on the corpora. The BOW is basically a frequency Counter, so I have written a function to get BOW from the corpora in my text_clean.py module.

In [14]:
# create clean corpus for bow approach
test_clean_token = clean_corpus(test_corpus, string_line=False)
train_clean_token = clean_corpus(train_corpus, string_line=False)

>>>> response cleaning initiated
--cleaning response #500 out of 898
>>>> response cleaning initiated
--cleaning response #500 out of 2186
--cleaning response #1000 out of 2186
--cleaning response #1500 out of 2186
--cleaning response #2000 out of 2186


In [15]:
# quick look at the word frequency
test_bow, test_word_freq = get_bow(test_clean_token)
train_bow, train_word_freq = get_bow(train_clean_token)

This corpus has 7126 key words, and the 10 most frequent words are: [('say', 2976), ('lt', 1112), ('share', 1067), ('dlrs', 921), ('company', 886), ('pct', 758), ('mln', 755), ('inc', 637), ('bank', 505), ('corp', 500)]
This corpus has 11042 key words, and the 10 most frequent words are: [('say', 7388), ('lt', 2802), ('share', 2306), ('dlrs', 2247), ('mln', 2172), ('pct', 2051), ('bank', 1983), ('company', 1942), ('inc', 1469), ('u', 1291)]


In [16]:
print(pd.DataFrame(test_bow).head())

   aabex  aame  aar  ab  abandon  abate  abatement  abboud  abegglen  abeles  \
0    NaN   NaN  NaN NaN      NaN    NaN        NaN     NaN       NaN     NaN   
1    NaN   NaN  NaN NaN      NaN    NaN        NaN     NaN       NaN     NaN   
2    NaN   NaN  NaN NaN      NaN    NaN        NaN     NaN       NaN     NaN   
3    NaN   NaN  NaN NaN      NaN    NaN        NaN     NaN       NaN     NaN   
4    NaN   NaN  NaN NaN      NaN    NaN        NaN     NaN       NaN     NaN   

     ...     zellerbach  zenex  zinn  zoete  zond  zondervan  zone  zoran  \
0    ...            NaN    NaN   NaN    NaN   NaN        NaN   NaN    NaN   
1    ...            NaN    NaN   NaN    NaN   NaN        NaN   NaN    NaN   
2    ...            NaN    NaN   NaN    NaN   NaN        NaN   NaN    NaN   
3    ...            NaN    NaN   NaN    NaN   NaN        NaN   NaN    NaN   
4    ...            NaN    NaN   NaN    NaN   NaN        NaN   NaN    NaN   

   zurich  zwermann  
0     NaN       NaN  
1     NaN   

The typical way to reduce the noise in BOW models is to use dimensionality reduction methods, such as Latent Semantic Indexing (LSA), Random Projections (RP), Latent Dirichlet Allocation (LDA), or Hierachical Dirichlet Process (HDP).

# Glimpse of Word2Vec Model
The alternative, of course, is to use the famous word2vec algorithm to generate continous numeric vectors. I will use gensim package to conduct this task. After I train the model with the reuters corpora, I get a dictionary that maps a numeric vector to each word that appears in the corpora.

In [17]:

# set up logging for gensim training
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model = gensim.models.Word2Vec(train_clean_token, min_count=1, workers=2)

2018-08-03 12:11:04,559 : INFO : collecting all words and their counts
2018-08-03 12:11:04,562 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-08-03 12:11:04,670 : INFO : collected 11042 word types from a corpus of 191770 raw words and 2186 sentences
2018-08-03 12:11:04,672 : INFO : Loading a fresh vocabulary
2018-08-03 12:11:04,751 : INFO : min_count=1 retains 11042 unique words (100% of original 11042, drops 0)
2018-08-03 12:11:04,752 : INFO : min_count=1 leaves 191770 word corpus (100% of original 191770, drops 0)
2018-08-03 12:11:04,882 : INFO : deleting the raw counts dictionary of 11042 items
2018-08-03 12:11:04,884 : INFO : sample=0.001 downsamples 40 most-common words
2018-08-03 12:11:04,886 : INFO : downsampling leaves estimated 169461 word corpus (88.4% of prior 191770)
2018-08-03 12:11:04,981 : INFO : estimated required memory for 11042 words and 100 dimensions: 14354600 bytes
2018-08-03 12:11:04,983 : INFO : resetting layer weights
2018-08-03

In [37]:
cbow_m1 = gensim.models.Word2Vec(train_clean_token, min_count=1, workers=2, window = 5, size=100)
cbow_m2 = gensim.models.Word2Vec(train_clean_token, min_count=5, workers=2, window = 5, size=100)
cbow_m3 = gensim.models.Word2Vec(train_clean_token, min_count=1, workers=2, window = 10, size=100)
cbow_m4 = gensim.models.Word2Vec(train_clean_token, min_count=1, workers=2, window = 5, size=300)

2018-08-03 12:14:04,568 : INFO : collecting all words and their counts
2018-08-03 12:14:04,571 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-08-03 12:14:04,679 : INFO : collected 11042 word types from a corpus of 191770 raw words and 2186 sentences
2018-08-03 12:14:04,680 : INFO : Loading a fresh vocabulary
2018-08-03 12:14:04,768 : INFO : min_count=1 retains 11042 unique words (100% of original 11042, drops 0)
2018-08-03 12:14:04,770 : INFO : min_count=1 leaves 191770 word corpus (100% of original 191770, drops 0)
2018-08-03 12:14:04,928 : INFO : deleting the raw counts dictionary of 11042 items
2018-08-03 12:14:04,930 : INFO : sample=0.001 downsamples 40 most-common words
2018-08-03 12:14:04,931 : INFO : downsampling leaves estimated 169461 word corpus (88.4% of prior 191770)
2018-08-03 12:14:05,020 : INFO : estimated required memory for 11042 words and 100 dimensions: 14354600 bytes
2018-08-03 12:14:05,021 : INFO : resetting layer weights
2018-08-03

In [46]:
cbow_m5 = gensim.models.Word2Vec(train_clean_token, min_count=1, workers=2, window = 30, size=100)

2018-08-03 12:18:41,729 : INFO : collecting all words and their counts
2018-08-03 12:18:41,732 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-08-03 12:18:41,838 : INFO : collected 11042 word types from a corpus of 191770 raw words and 2186 sentences
2018-08-03 12:18:41,840 : INFO : Loading a fresh vocabulary
2018-08-03 12:18:42,158 : INFO : min_count=1 retains 11042 unique words (100% of original 11042, drops 0)
2018-08-03 12:18:42,160 : INFO : min_count=1 leaves 191770 word corpus (100% of original 191770, drops 0)
2018-08-03 12:18:42,282 : INFO : deleting the raw counts dictionary of 11042 items
2018-08-03 12:18:42,285 : INFO : sample=0.001 downsamples 40 most-common words
2018-08-03 12:18:42,287 : INFO : downsampling leaves estimated 169461 word corpus (88.4% of prior 191770)
2018-08-03 12:18:42,402 : INFO : estimated required memory for 11042 words and 100 dimensions: 14354600 bytes
2018-08-03 12:18:42,403 : INFO : resetting layer weights
2018-08-03

In [47]:
print(cbow_m1.wv.similarity('love', 'like'))
print(cbow_m2.wv.similarity('love', 'like'))
print(cbow_m3.wv.similarity('love', 'like'))
print(cbow_m4.wv.similarity('love', 'like'))
print(cbow_m5.wv.similarity('love', 'like'))

0.894432118604
0.888915671319
0.907082319364
0.895234299138
0.641843491056


In [48]:
print(cbow_m1.wv.similarity('men', 'women'))
#print(cbow_m2.wv.similarity('men', 'women'))
print(cbow_m3.wv.similarity('men', 'women'))
print(cbow_m4.wv.similarity('men', 'women'))
print(cbow_m5.wv.similarity('men', 'women'))

0.883191289861
0.0930611731805
0.94528804297
0.799890008917


In [49]:
ng_m1 = gensim.models.Word2Vec(train_clean_token, min_count=1, workers=2, window = 5, size=100, sg=1)
ng_m2 = gensim.models.Word2Vec(train_clean_token, min_count=5, workers=2, window = 5, size=100, sg=1)
ng_m3 = gensim.models.Word2Vec(train_clean_token, min_count=1, workers=2, window = 10, size=100, sg=1)
ng_m4 = gensim.models.Word2Vec(train_clean_token, min_count=1, workers=2, window = 5, size=300, sg=1)
ng_m5 = gensim.models.Word2Vec(train_clean_token, min_count=1, workers=2, window = 30, size=300, sg=1)

2018-08-03 12:20:38,115 : INFO : collecting all words and their counts
2018-08-03 12:20:38,116 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-08-03 12:20:38,214 : INFO : collected 11042 word types from a corpus of 191770 raw words and 2186 sentences
2018-08-03 12:20:38,216 : INFO : Loading a fresh vocabulary
2018-08-03 12:20:38,296 : INFO : min_count=1 retains 11042 unique words (100% of original 11042, drops 0)
2018-08-03 12:20:38,298 : INFO : min_count=1 leaves 191770 word corpus (100% of original 191770, drops 0)
2018-08-03 12:20:38,419 : INFO : deleting the raw counts dictionary of 11042 items
2018-08-03 12:20:38,422 : INFO : sample=0.001 downsamples 40 most-common words
2018-08-03 12:20:38,423 : INFO : downsampling leaves estimated 169461 word corpus (88.4% of prior 191770)
2018-08-03 12:20:38,507 : INFO : estimated required memory for 11042 words and 100 dimensions: 14354600 bytes
2018-08-03 12:20:38,509 : INFO : resetting layer weights
2018-08-03

In [53]:
ng_m6 = gensim.models.Word2Vec(train_clean_token, min_count=1, workers=2, window = 30, size=100, sg=1)

2018-08-03 12:27:40,282 : INFO : collecting all words and their counts
2018-08-03 12:27:40,285 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-08-03 12:27:40,391 : INFO : collected 11042 word types from a corpus of 191770 raw words and 2186 sentences
2018-08-03 12:27:40,393 : INFO : Loading a fresh vocabulary
2018-08-03 12:27:40,491 : INFO : min_count=1 retains 11042 unique words (100% of original 11042, drops 0)
2018-08-03 12:27:40,493 : INFO : min_count=1 leaves 191770 word corpus (100% of original 191770, drops 0)
2018-08-03 12:27:40,613 : INFO : deleting the raw counts dictionary of 11042 items
2018-08-03 12:27:40,616 : INFO : sample=0.001 downsamples 40 most-common words
2018-08-03 12:27:40,617 : INFO : downsampling leaves estimated 169461 word corpus (88.4% of prior 191770)
2018-08-03 12:27:40,714 : INFO : estimated required memory for 11042 words and 100 dimensions: 14354600 bytes
2018-08-03 12:27:40,716 : INFO : resetting layer weights
2018-08-03

In [54]:
print(ng_m1.wv.similarity('men', 'women'))
#print(ng_m2.wv.similarity('men', 'women'))
print(ng_m3.wv.similarity('men', 'women'))
print(ng_m4.wv.similarity('men', 'women'))
print(ng_m5.wv.similarity('men', 'women'))
print(ng_m6.wv.similarity('men', 'women'))

0.960223762102
0.616894966563
0.960637996297
0.65993672376
0.621251033744


With the model, I can get the numeric representation of any word that's included in the model's dictionary, and even quickly calculate the "similarity" between two words.

In [18]:
print("Printing the vector of 'inc': {} ...".format(model['inc'][:10]))
print("Printing the similarity between 'inc' and 'love': {}"\
      .format(model.wv.similarity('inc', 'love')))
print("Printing the similarity between 'inc' and 'company': {}"\
      .format(model.wv.similarity('inc', 'company')))

Printing the vector of 'inc': [ 0.69633245  1.34032726  0.32753372 -0.28197432 -1.15196598 -0.1479332
 -1.96487784  0.59153831 -0.57587731  0.36454782] ...
Printing the similarity between 'inc' and 'love': 0.8710930305539258
Printing the similarity between 'inc' and 'company': 0.9289818467307422


/home/sayali/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [19]:
model.wv.similarity('man', 'women')

0.95367470501715346

In [20]:
model.wv.similarity('king', 'women')

0.96089887955037723

In [21]:
model.wv.similarity('king', 'man')

0.99417811611921247

In [22]:
model.wv.similarity('like', 'love')

0.89148801691870572

In [23]:
print(len(model[model.wv.index2word[0]]))

100


/home/sayali/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [24]:
def get_doc_matrix(w2v_model, corpora_token):
    """
    Function to aggregate document vector from a built gensim w2v model that
    calculates the vector mean based on vector representation of words in the
    document
    -----PARAMETERS-----
    w2v_model: a gensim.models.Word2Vec object that can return a numeric array
        when queried with w2v_model['word']
    corpora_token: a list of sentence in list form, e.g. [['sentence','one'],
        ['sentence','two'],...]
    -----OUTPUT-----
    returned object (text_matrix) is a numpy.ndarray with the shape
    (len(corpora_token), word_vector_length)
    """
    word_vector_length = len(w2v_model[w2v_model.wv.index2word[0]])  # get word vector length
    text_matrix = np.zeros((len(corpora_token), word_vector_length))
    for i in range(len(corpora_token)):
        text_vector = np.zeros(word_vector_length)
        for j in range(len(corpora_token[i])):
            try:
                text_vector += w2v_model[corpora_token[i][j]]
            except:
                pass
            if j == len(corpora_token[i]) - 1:
                text_vector = text_vector / len(corpora_token[i])
        text_matrix[i][:] = text_vector
    return text_matrix

In [25]:
def generate_doc_vector_matrix(wv_model, corpus):
    word_vector_length = len(wv_model[wv_model.wv.index2word[0]])
    text_matrix = np.zeros((len(corpus),len(wv_model[wv_model.wv.index2word[0]])))
    for doc_no in range(len(corpus)):
        text_vector = np.zeros(word_vector_length)
        for word_no in range(len(corpus[doc_no])):
            try:
                text_vector += wv_model[corpus[doc_no][word_no]]
            except:
                pass
            if word_no == len(corpus[doc_no]) - 1:       # if you reach at last word of document
                text_vector = text_vector / len(corpus[doc_no]) # Create average by dividing sum
        text_matrix[doc_no][:] = text_vector
    return text_matrix

In [26]:
generate_doc_vector_matrix(model, test_clean_token)

/home/sayali/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app
/home/sayali/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()
/home/sayali/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


array([[ 0.17918864,  0.61735579, -0.12018448, ...,  0.10357096,
         0.18109203,  0.03014295],
       [ 0.1254951 ,  0.84262503, -0.25072033, ...,  0.21809811,
         0.26649487,  0.00959609],
       [ 0.18357663,  0.70173899, -0.04382254, ...,  0.09582808,
         0.26255332,  0.12280407],
       ..., 
       [ 0.17405347,  0.70252212, -0.17179839, ...,  0.16586508,
         0.16602121,  0.01405772],
       [ 0.14174666,  0.7925582 , -0.05796555, ...,  0.17251987,
         0.37840594,  0.16619108],
       [ 0.2563348 ,  0.73224282, -0.21198497, ...,  0.16326701,
         0.07083301, -0.04412346]])

In [27]:
test_matrix = get_doc_matrix(model, test_clean_token)
train_matrix = get_doc_matrix(model, train_clean_token)


/home/sayali/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/home/sayali/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:21: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [28]:
print(pd.DataFrame(train_matrix).head())

         0         1         2         3         4         5         6   \
0  0.145580  0.820419 -0.046329 -0.060312 -0.700513  0.229105 -0.652282   
1  0.187050  0.885292 -0.019713 -0.080886 -0.761505  0.189836 -0.761482   
2  0.260295  0.671416 -0.106457 -0.249679 -0.534678  0.074275 -0.359636   
3  0.301887  0.761861  0.085036 -0.209179 -0.615830  0.031189 -0.667108   
4  0.102003  0.864999 -0.055664 -0.132975 -0.734411  0.358447 -0.558825   

         7         8         9     ...           90        91        92  \
0  0.103089 -0.037491  0.044101    ...    -0.316500 -1.095399 -0.631698   
1  0.148969 -0.068057  0.070770    ...    -0.381748 -1.151074 -0.669432   
2  0.282031  0.043433 -0.037342    ...    -0.121360 -0.768783 -0.464567   
3  0.262176 -0.092785  0.097573    ...    -0.393109 -0.955765 -0.448833   
4  0.096909  0.053771 -0.005729    ...    -0.295702 -1.291409 -0.601369   

         93        94        95        96        97        98        99  
0  0.465621 -0.179941  0

In [29]:
print(pd.DataFrame(test_matrix).head())

         0         1         2         3         4         5         6   \
0  0.179189  0.617356 -0.120184 -0.178332 -0.489537  0.119097 -0.343348   
1  0.125495  0.842625 -0.250720 -0.206060 -0.679819  0.337556 -0.239724   
2  0.183577  0.701739 -0.043823 -0.163661 -0.579389  0.164973 -0.443522   
3  0.052820  1.006113 -0.273856 -0.322318 -0.781469  0.656680 -0.232314   
4  0.180800  0.691567  0.006618 -0.131686 -0.595601  0.185493 -0.586476   

         7         8         9     ...           90        91        92  \
0  0.208368  0.027998 -0.016479    ...    -0.134513 -0.747695 -0.448916   
1  0.171765  0.197963 -0.133512    ...    -0.012870 -1.061079 -0.617072   
2  0.187899  0.020348  0.005562    ...    -0.212436 -0.903332 -0.477231   
3  0.106958  0.381380 -0.267700    ...     0.060949 -1.612309 -0.724327   
4  0.202047 -0.048276  0.034002    ...    -0.300576 -0.977935 -0.508895   

         93        94        95        96        97        98        99  
0  0.135658 -0.023119  0

### Label Binerizer

In [30]:
all_labels = []
for i in train_labels:
    all_labels.extend(i)
for i in test_labels:
    all_labels.extend(i)

In [31]:
mlb = MultiLabelBinarizer(classes=list(set(all_labels)))
train_labels_bin = mlb.fit_transform(train_labels)
test_labels_bin = mlb.transform(test_labels)

In [32]:
# Classifier
classifier = OneVsRestClassifier(LinearSVC(random_state=42))
classifier.fit(train_matrix, train_labels_bin)
y_pred = classifier.predict(test_matrix) 
y_pred

/home/sayali/anaconda3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 2 is present in all training examples.
  str(classes[c]))
/home/sayali/anaconda3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 6 is present in all training examples.
  str(classes[c]))
/home/sayali/anaconda3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 13 is present in all training examples.
  str(classes[c]))
/home/sayali/anaconda3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 14 is present in all training examples.
  str(classes[c]))
/home/sayali/anaconda3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 22 is present in all training examples.
  str(classes[c]))
/home/sayali/anaconda3/lib/python3.5/site-packages/sklearn/multiclass.py:76: UserWarning: Label not 26 is present in all training examples.
  str(classes[c]))
/home/sayali/anaconda3/lib/python3.5/site-packag

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [33]:
predictions = mlb.inverse_transform(y_pred)

In [34]:
confusion = pd.DataFrame({"Predicted": test_labels, "Actual":test_labels })

In [35]:
confusion

,Actual,Predicted
0,[acq],[acq]
1,"[interest, money-fx]","[interest, money-fx]"
2,"[acq, copper]","[acq, copper]"
3,"[interest, money-fx]","[interest, money-fx]"
4,[acq],[acq]
5,[acq],[acq]
6,"[interest, money-fx]","[interest, money-fx]"
7,[acq],[acq]
8,[acq],[acq]
9,[acq],[acq]


In [36]:
def evaluate(test_labels, predictions):
    precision = precision_score(test_labels, predictions, average='micro')
    recall = recall_score(test_labels, predictions, average='micro')
    f1 = f1_score(test_labels, predictions, average='micro')
    print("Micro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

    precision = precision_score(test_labels, predictions, average='macro')
    recall = recall_score(test_labels, predictions, average='macro')
    f1 = f1_score(test_labels, predictions, average='macro')
    print("Macro-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))
    
    precision = precision_score(test_labels, predictions, average='samples')
    recall = recall_score(test_labels, predictions, average='samples')
    f1 = f1_score(test_labels, predictions, average='samples')
    print("samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

In [75]:
evaluate(y_pred, test_labels_bin)

Micro-average quality numbers
Precision: 0.8833, Recall: 0.9851, F1-measure: 0.9314
Macro-average quality numbers
Precision: 0.0627, Recall: 0.0806, F1-measure: 0.0681
samples-average quality numbers
Precision: 0.9421, Recall: 0.9900, F1-measure: 0.9548


/home/sayali/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/sayali/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/home/sayali/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/sayali/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [76]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [78]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(train_clean_string)]

In [79]:
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

2018-08-02 20:50:54,522 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2018-08-02 20:50:54,594 : INFO : collecting all words and their counts
2018-08-02 20:50:54,596 : WARNING : Each 'words' should be a list of words (usually unicode strings). First 'words' here is instead plain <class 'str'>.
2018-08-02 20:50:54,600 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2018-08-02 20:50:54,909 : INFO : collected 27 word types and 2186 unique tags from a corpus of 2186 examples and 1305589 words
2018-08-02 20:50:54,911 : INFO : Loading a fresh vocabulary
2018-08-02 20:50:54,913 : INFO : min_count=1 retains 27 unique words (100% of original 27, drops 0)
2018-08-02 20:50:54,915 : INFO : min_count=1 leaves 1305589 word corpus (100% of original 1305589, drops 0)
2018-08-02 20:50:54,922 : INFO : deleting the raw counts dictionary of 27 items
2018-08-02 20:50:54,924 : INFO : sample=0.001 downsamples 24 most-common words
2018-08-02 

In [80]:
model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [82]:
model.infer_vector(["system", "response"])

array([ 0.06562854, -0.0525164 ,  0.05182675, -0.02825312, -0.03992648], dtype=float32)

In [87]:
model.docvecs.most_similar()

2018-08-02 20:55:18,084 : INFO : precomputing L2-norms of doc weight vectors


ValueError: cannot compute similarity with no input